<a href="https://colab.research.google.com/github/ZDDATApol4/prediction-political-preferences/blob/main/GUI_projekt_ko%C5%84cowy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title SPRAWDŹ SWOJE POLITYCZNE PREFERENCJE

ID = "JasKapela" #@param ["", ""] {allow-input: true}


In [2]:
import tweepy
import pandas as pd
import time

# Credentials

consumer_key = "5MzB6K9I3B0hVF0BOs2UjybSz"
consumer_secret = "hiPjP10G7bTuomHflC1vCWzwcdsAB444WkE0MkSZwGGfALoxBS"
access_token = "1437020925040513029-s6Dqtv9lPyufnlKtzm9tF3BQEQ8FCA"
access_token_secret = "D9SUsW12b1ya9LuWPNNmGmU0NhKIJzoeFfHsIy4xgdJ7x"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

tweets = []

def username_tweets_to_csv(username,count):
    try:      
        # Creation of query method using parameters
        #tweets = tweepy.Cursor(api.user_timeline,id=username).items(count)
        tweets = tweepy.Cursor(api.user_timeline,id=username, since='01-08-2021').items(count)

        # Pulling information from tweets iterable object
        tweets_list = [[tweet.user.screen_name, tweet.created_at, tweet.id, tweet.text] for tweet in tweets]

        # Creation of dataframe from tweets list
        # Add or remove columns as you remove tweet information
        tweets_df = pd.DataFrame(tweets_list,columns=['Author', 'Datetime', 'Tweet Id', 'Text'])
        

        # Converting dataframe to CSV 
        #tweets_df.to_csv('{}.csv'.format(username), sep=',', index = False)
        return tweets_df

    except BaseException as e:
          print('failed on_status,',str(e))
          time.sleep(3)


count = 1000
data = username_tweets_to_csv(ID, count)

In [3]:
data

,Author,Datetime,Tweet Id,Text
0,JasKapela,2021-09-23 18:01:15,1441100363830939649,"@juper93 Wolałbym zakonnicę, ale w sumie chętn..."
1,JasKapela,2021-09-23 17:58:50,1441099755094888449,@e_pjoter @Greenpeace_PL Taka prawda. Ludzinę ...
2,JasKapela,2021-09-23 17:48:02,1441097034723782657,RT @LicznikKar: #Turów Stan na 22-09-2021 \n🇪🇺...
3,JasKapela,2021-09-23 17:47:29,1441096898455097367,@Rubinho_RPK Mnie nie
4,JasKapela,2021-09-23 17:47:12,1441096826229194769,"RT @HebanMaja: Jak to jest, że życie ludzkie m..."
...,...,...,...,...
995,JasKapela,2021-05-19 13:10:36,1395003961086726144,"@patryk_sniadala @K_Stanowski dzięki, już odpi..."
996,JasKapela,2021-05-19 13:05:39,1395002714464133125,@grz79 @komediant68 @PiotrStrzembosz @miloszlo...
997,JasKapela,2021-05-19 13:04:35,1395002443365236736,@grz79 @komediant68 @PiotrStrzembosz @miloszlo...
998,JasKapela,2021-05-19 12:23:24,1394992080976588800,@K_Stanowski nie wchodziłem przez weekend na t...


Przygotowanie danych do predykcji

In [4]:
!pip install spacy==3

     |████████████████████████████████| 12.7 MB 222 kB/s 
     |████████████████████████████████| 9.1 MB 29.4 MB/s 
     |████████████████████████████████| 623 kB 39.1 MB/s 
     |████████████████████████████████| 456 kB 55.1 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [5]:
!python -m spacy download pl_core_news_sm

     |████████████████████████████████| 58.2 MB 52 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_sm')


In [6]:
import keras
import spacy
import numpy as np

from keras_preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm

nlp = spacy.load('pl_core_news_sm')

In [7]:
import pandas as pd

In [8]:
import string
from string import digits
import re

In [9]:
data = data[data['Datetime'] >= pd.to_datetime('2021-08-01')]
test_text_corpus = data['Text']

# Remove @replies from source and target text
regex = r'(@\w+\s)+'
test_text_corpus = test_text_corpus.apply(lambda x: re.sub(regex, '', x))

# Remove URLS from source and target text
regex = r'https?:\/\/.*[\r\n]*'
test_text_corpus = test_text_corpus.apply(lambda x: re.sub(regex, '', x, flags=re.MULTILINE))

# convert source and target text to Lowercase 
test_text_corpus = test_text_corpus.apply(lambda x: x.lower())

# Remove quotes from source and target text
test_text_corpus = test_text_corpus.apply(lambda x: re.sub("'", '', x))

# create a set of all special characters
special_characters= set(string.punctuation)
# Remove all the special characters
test_text_corpus = test_text_corpus.apply(lambda x: ''.join(char1 for char1 in x if char1 not in special_characters))

# Remove digits from source and target sentences
num_digits= str.maketrans('','', digits)
test_text_corpus = test_text_corpus.apply(lambda x: x.translate(num_digits))

In [10]:
def lemmatize_text(text_corpus):
  lemmatized_text = []
  for tweet in tqdm(text_corpus):
    doc = nlp(tweet)
    doc_lemmatized = []
    for x in doc:
      if x.is_punct == False and x.is_stop == False:
        doc_lemmatized.append(x.lemma_)
    lemmatized_text.append(doc_lemmatized)
  return lemmatized_text

In [11]:
test_lemmatized = lemmatize_text(test_text_corpus)

100%|██████████| 376/376 [00:04<00:00, 84.80it/s]


In [12]:
def tokenize(num_words, lemmatized_text, maxlen):
  tokenizer = Tokenizer(num_words=num_words)
  tokenizer.fit_on_texts(lemmatized_text)
  sequences = tokenizer.texts_to_sequences(lemmatized_text)
  padded = pad_sequences(sequences, maxlen=maxlen)
  print(padded.shape)
  return padded

In [13]:
test_padded = tokenize(num_words=50000, lemmatized_text=test_lemmatized, maxlen=20)

(376, 20)


Wczytanie modelu

In [14]:
!git clone https://github.com/ZDDATApol4/prediction-political-preferences

Cloning into 'prediction-political-preferences'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 49 (delta 23), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (49/49), done.


In [15]:
from keras.models import load_model

In [18]:
loaded_model = load_model('prediction-political-preferences/test_model_for_dash.h5')
#loaded_model = load_model('prediction-political-preferences/pre_trained_nkjp+wiki_model.h5')

In [19]:
predicted = loaded_model.predict(test_padded)
predicted.shape

(376, 5)

In [22]:
df = pd.DataFrame(predicted, columns=['KO', 'Konfederacja', 'Lewica', 'PSL', 'PiS'])
df.mean()

KO              0.266650
Konfederacja    0.214942
Lewica          0.240687
PSL             0.159778
PiS             0.117943
dtype: float32

Wykres kołowy politycznych preferencji

In [68]:
ID

'JasKapela'

In [97]:
from plotly.offline import iplot

pd.plotting.register_matplotlib_converters()

pie = df.mean()
labels = ['KO', 'Konfederacja', 'Lewica', 'PSL', 'PiS']
fig = {
    'data': [
      {
       'values': pie,
       'labels': labels,
       'domain': {'x': [0, .5]},
       'name': 'Liczba studentów procentowo',
       'hoverinfo': 'label+percent+name',
       'hole': .3,
       'type': 'pie'
       }, ],
     'layout': {
         'legend_x': 0.5,
         'legend_y': 0.5,
         'title': f'Preferencje polityczne użytkownika "{ID}"',
         'title_x': 0.1,
         'title_y': 0.9,
     }  
}
iplot(fig)

Predykcja danych z modelu i opracowanie danych

Dashboard

In [ ]:
pip install dash pandas

     |████████████████████████████████| 7.3 MB 28.9 MB/s 
     |████████████████████████████████| 23.9 MB 14 kB/s 
     |████████████████████████████████| 357 kB 51.8 MB/s 
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3821 sha256=83b1d8d8763d024a61c8d06f0eec1cf8bb3f5cdd68ebde862d11938dab78d0ef
  Stored in directory: /root/.cache/pip/wheels/00/f9/c7/1a6437d794ed753ea9bc9079e761d4fc803a1f1f5d3697b9ec
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4089 sha256=bf9af2601f7d1a7da7690cbce274b48cccbb649d7849e140d0e5ce7c883be62f
  Stored in directory: /root/.cache/pip/wheels/ec/6b/81/05aceabd8b27f724e2c96784016287cc1bfbc349ebfda451de
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl size=3911 sha256=cf1c2183b53f3a39760206d6ebb841c1c066bef4ffe1b180e805ac9deb299d71
  Stored in directory: /root/.cache/pip/wheels/85/5d/4e/7c276b57992951dbe770bf5caad6448d0539c510663aef

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd

external_stylesheets = [
    'https://codepen.io/chriddyp/pen/bWLwgP.css'
]

# Create Dash app
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
# Expose Flask instance
server = app.server

# Alternative version, using flask server directly (add import flask in imports)
# server = flask.Flask(__name__)
# app = dash.Dash(__name__, external_stylesheets=external_stylesheets, server=server)

# Assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})

fig = px.bar(
    df, 
    x="Fruit", y="Amount", color="City", barmode="group"
)

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python. Customized right here!
    '''),
    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])

# Only for running on development mode
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
